# Fruitvale Race Census Data Exploration


In [2]:
#Import packages
import pandas as pd
import matplotlib as mpl
#import matplotlib.pyplot as plt
import numpy as np
import math
import os
from openpyxl import load_workbook


In [3]:
#set directories
os.chdir("/Users/briangoggin/Dropbox/CP 201A/Fruitvale")
cwd = os.getcwd()
root = cwd+"/Raw Data/" #root is directory to raw files
root

'/Users/briangoggin/Dropbox/CP 201A/Fruitvale/Raw Data/'

In [4]:
#import raw data file

race = root+"ACS_14_5YR_B03002_with_ann.csv"
education = root+"ACS_14_5YR_B15003_with_ann.csv"
tenure = root+"ACS_14_5YR_B25003_with_ann.csv"
commute_mode = root+"ACS_14_5YR_B08101_with_ann.csv"
poverty = root+"ACS_14_5YR_B17001_with_ann.csv"
employment = root+"ACS_14_5YR_B23025_with_ann.csv"
vehicles = root+"ACS_14_5YR_B08203_with_ann.csv"
time = root+"ACS_14_5YR_B08134_with_ann.csv"
trans_pov = root+"ACS_14_5YR_B08122_with_ann.csv"
age = root+"ACS_14_5YR_S0101_with_ann.csv"
income = root+"ACS_14_5YR_B19001_with_ann.csv"

race

'/Users/briangoggin/Dropbox/CP 201A/Fruitvale/Raw Data/ACS_14_5YR_B03002_with_ann.csv'

In [5]:
#Import Datasets into dataframes

#first, define lists of file paths and Fruitvale Census Tracts
datasets = [race, education, tenure, commute_mode, poverty]
Fruitvale = ['06001406100', '06001406201', '06001406202', '06001406300', '06001406500', '06001407101', '06001407102', '06001407200', '06001407300', '06001407400', '06001407500', '06001407600']

#Define function for import and standard ACS cleaning operations
def import_census(file):
    df = pd.read_csv(file)
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df['Fruitvale'] = df['Id2'].isin(Fruitvale)
    #the following 5 lines create duplicate observations of Fruitvale to include them in county averages
    df2 = df[df['Fruitvale'] == True]
    df2.drop('Fruitvale', axis = 1)
    df2.is_copy = False
    df2['Fruitvale'] = False
    df = df.append(df2)
    return df

#import all the data
race = import_census(race)
education = import_census(education)
tenure = import_census(tenure)
commute_mode = import_census(commute_mode)
poverty = import_census(poverty)
employment = import_census(employment)
vehicles = import_census(vehicles)
time = import_census(time)
trans_pov = import_census(trans_pov)
age = import_census(age)
income = import_census(income)
income.head()

,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,"Estimate; Total: - Less than $10,000","Margin of Error; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Margin of Error; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999",...,"Margin of Error; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Margin of Error; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Margin of Error; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Margin of Error; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more","Margin of Error; Total: - $200,000 or more",Fruitvale
1,1400000US06001400100,06001400100,"Census Tract 4001, Alameda County, California",1300,66,32,27,11,17,39,...,56,147,64,72,40,167,67,530,100,False
2,1400000US06001400200,06001400200,"Census Tract 4002, Alameda County, California",815,48,15,13,0,12,10,...,30,110,40,96,42,107,52,251,48,False
3,1400000US06001400300,06001400300,"Census Tract 4003, Alameda County, California",2510,95,67,51,308,140,66,...,88,175,91,179,90,224,97,465,132,False
4,1400000US06001400400,06001400400,"Census Tract 4004, Alameda County, California",1812,81,71,45,58,55,60,...,86,245,114,185,66,234,80,226,78,False
5,1400000US06001400500,06001400500,"Census Tract 4005, Alameda County, California",1590,78,38,32,138,72,0,...,91,231,84,62,39,106,49,68,32,False


In [6]:
#Define function for proportions MOE
def prop_MOE(numerator, denominator, num_moe, den_moe):
    return ((num_moe**2 - (((numerator/denominator)**2)*(den_moe**2))))**(1/2)/denominator



In [7]:
#############################
##Section 1: Commute Mode
#############################

#Convert all commute mode variables to integers
commute_mode['total_pop'] = commute_mode["Estimate; Total:"].astype(int)
commute_mode['total_pop_moe'] = commute_mode["Margin of Error; Total:"].astype(int)
commute_mode['car_alone'] = commute_mode["Estimate; Total: - Car, truck, or van - drove alone:"].astype(int)
commute_mode['car_alone_moe'] = commute_mode["Margin of Error; Total: - Car, truck, or van - drove alone:"].astype(int)
commute_mode['publict'] = commute_mode["Estimate; Total: - Public transportation (excluding taxicab):"].astype(int)
commute_mode['publict_moe'] = commute_mode["Margin of Error; Total: - Public transportation (excluding taxicab):"].astype(int)
commute_mode['carpool'] = commute_mode["Estimate; Total: - Car, truck, or van - carpooled:"].astype(int)
commute_mode['carpool_moe'] = commute_mode["Margin of Error; Total: - Car, truck, or van - carpooled:"].astype(int)
commute_mode['walk'] = commute_mode["Estimate; Total: - Walked:"].astype(int)
commute_mode['walk_moe'] = commute_mode["Margin of Error; Total: - Walked:"].astype(int)
commute_mode['taxi_bike'] = commute_mode["Estimate; Total: - Taxicab, motorcycle, bicycle, or other means:"].astype(int)
commute_mode['taxi_bike_moe'] = commute_mode["Margin of Error; Total: - Taxicab, motorcycle, bicycle, or other means:"].astype(int)
commute_mode['home'] = commute_mode["Estimate; Total: - Worked at home:"].astype(int)
commute_mode['home_moe'] = commute_mode["Margin of Error; Total: - Worked at home:"].astype(int)



In [8]:
#list moes to transform
list = ['total_pop_moe', 'publict_moe', 'car_alone_moe', 
               'carpool_moe', 'walk_moe', 'taxi_bike_moe', 
              'home_moe']

#convert MOEs to 95% confidence level
def convert(column):
    return column*(1.96/1.645)

for item in list:
    commute_mode[item] = convert(commute_mode[item])

In [9]:
#Create Commute standard MOEs

#square MOEs for aggregate MOEs
commute_mode['total_pop_moe2'] = (commute_mode['total_pop_moe'])**2
commute_mode['publict_moe2'] = (commute_mode['publict_moe'])**2
commute_mode['car_alone_moe2'] = (commute_mode['car_alone_moe'])**2
commute_mode['carpool_moe2'] = (commute_mode['carpool_moe'])**2
commute_mode['walk_moe2'] = (commute_mode['walk_moe'])**2
commute_mode['taxi_bike_moe2'] = (commute_mode['taxi_bike_moe'])**2
commute_mode['home_moe2'] = (commute_mode['home_moe'])**2

#create 'other' category, its moe, and its moe squared
commute_mode['other'] = commute_mode['walk'] + commute_mode['taxi_bike'] + commute_mode['home']
commute_mode['other_moe'] = (commute_mode['walk_moe2'] + commute_mode['taxi_bike_moe2'] + commute_mode['home_moe2'])**(1/2) 
commute_mode['other_moe2'] = (commute_mode['other_moe'])**2


#list data to include in sum
commutes = ['total_pop', 'publict', 'car_alone', 'carpool', 'other',
            'total_pop_moe2', 'publict_moe2', 'car_alone_moe2', 'carpool_moe2', 'other_moe2']

#Sum data 
excommutes = commute_mode.groupby('Fruitvale')[commutes].sum()

excommutes['total_pop_moe'] = np.sqrt(excommutes['total_pop_moe2'])
excommutes['pct_transit'] = (excommutes['publict']/excommutes['total_pop'])
excommutes['publict_moe'] = np.sqrt(excommutes['publict_moe2'])
excommutes['pct_car_alone'] = (excommutes['car_alone']/excommutes['total_pop'])
excommutes['car_alone_moe'] = np.sqrt(excommutes['car_alone_moe2'])
excommutes['pct_carpool'] = (excommutes['carpool']/excommutes['total_pop'])
excommutes['carpool_moe'] = np.sqrt(excommutes['carpool_moe2'])
excommutes['pct_other'] = (excommutes['other']/excommutes['total_pop'])
excommutes['other_moe'] = np.sqrt(excommutes['other_moe2'])

excommutes

,total_pop,publict,car_alone,carpool,other,total_pop_moe2,publict_moe2,car_alone_moe2,carpool_moe2,other_moe2,total_pop_moe,pct_transit,publict_moe,pct_car_alone,car_alone_moe,pct_carpool,carpool_moe,pct_other,other_moe
Fruitvale,,,,,,,,,,,,,,,,,,,
False,701525,89723,451989,71593,88220,3.399327e+07,6.833781e+06,2.467692e+07,6.139492e+06,6.876430e+06,5830.374441,0.127897,2614.150197,0.644295,4967.586934,0.102053,2477.799745,0.125755,2622.294851
True,22470,4526,11418,3493,3033,1.883471e+06,4.471263e+05,8.116604e+05,3.640415e+05,4.082933e+05,1372.396114,0.201424,668.675040,0.508144,900.921972,0.155452,603.358495,0.134980,638.978315


In [10]:
#generate proportion MOEs
excommutes['pct_transit_moe'] = prop_MOE(excommutes['publict'], excommutes['total_pop'], excommutes['publict_moe'], excommutes['total_pop_moe'])
excommutes['pct_car_alone_moe'] = prop_MOE(excommutes['car_alone'], excommutes['total_pop'], excommutes['car_alone_moe'], excommutes['total_pop_moe'])
excommutes['pct_carpool_moe'] = prop_MOE(excommutes['carpool'], excommutes['total_pop'], excommutes['carpool_moe'], excommutes['total_pop_moe'])
excommutes['pct_other_moe'] = prop_MOE(excommutes['other'], excommutes['total_pop'], excommutes['other_moe'], excommutes['total_pop_moe'])


In [11]:
#set variables to keep 
keep_vars = ['pct_car_alone', 'pct_transit', 
             'pct_carpool', 'pct_other', 
             'pct_car_alone_moe', 'pct_transit_moe', 
             'pct_carpool_moe', 'pct_other_moe']

excommutes = excommutes[keep_vars]

#change everything to rounded percents
excommutes = excommutes.multiply(100).round(2)
excommutes

,pct_car_alone,pct_transit,pct_carpool,pct_other,pct_car_alone_moe,pct_transit_moe,pct_carpool_moe,pct_other_moe
Fruitvale,,,,,,,,
False,64.43,12.79,10.21,12.58,0.46,0.36,0.34,0.36
True,50.81,20.14,15.55,13.50,2.54,2.71,2.51,2.72


In [12]:
#export data to csv
export_path = cwd+'/Output/python_output/commutes.csv'
excommutes = excommutes.to_csv(export_path)

In [113]:
#############################
##Section 2: Employment
#############################

#Convert all employment variables to integers
employment['pop'] = employment["Estimate; Total:"].astype(int)
employment['pop_moe'] = employment["Margin of Error; Total:"].astype(int)
employment['in_labor'] = employment['Estimate; In labor force:'].astype(int)
employment['in_labor_moe'] = employment['Margin of Error; In labor force:'].astype(int)
employment['civ_employed'] = employment["Estimate; In labor force: - Civilian labor force: - Employed"].astype(int)
employment['civ_employed_moe'] = employment["Margin of Error; In labor force: - Civilian labor force: - Employed"].astype(int)
employment['armed'] = employment["Estimate; In labor force: - Armed Forces"].astype(int)
employment['armed_moe'] = employment["Margin of Error; In labor force: - Armed Forces"].astype(int)
employment['unemployed'] = employment["Estimate; In labor force: - Civilian labor force: - Unemployed"].astype(int)
employment['unemployed_moe'] = employment["Margin of Error; In labor force: - Civilian labor force: - Unemployed"].astype(int)
employment['not_in_labor'] = employment["Estimate; Not in labor force"].astype(int)
employment['not_in_labor_moe'] = employment["Margin of Error; Not in labor force"].astype(int)



In [114]:
#Create employment standard MOEs

moes = ['pop_moe', 'in_labor_moe', 'civ_employed_moe', 'armed_moe', 
               'unemployed_moe', 'not_in_labor_moe']

#convert MOEs to 95% confidence level
def convert(column):
    return column*(1.96/1.645)

#square columns
def square(column):
    return column**2

for item in moes:
    employment[item] = convert(employment[item])
    employment[item+'2']= square(employment[item])

#create 'total empoyed' category, its moe, and its moe squared
employment['total_employed'] = employment['civ_employed'] + employment['armed']
employment['total_employed_moe'] = (employment['civ_employed_moe2'] + employment['armed_moe2'])**(1/2) 
employment['total_employed_moe2'] = (employment['total_employed_moe'])**2




In [115]:
#list data to include in sum
employments = ['pop', 'total_employed', 'unemployed', 'not_in_labor', 'in_labor', 
            'pop_moe2', 'total_employed_moe2', 'unemployed_moe2', 'not_in_labor_moe2', 'in_labor_moe2']


In [116]:
#Sum data 
exemployment = employment.groupby('Fruitvale')[employments].sum()

#new MOEs
exemployment['pop_moe'] = (exemployment['pop_moe2'])**(1/2)
exemployment['total_employed_moe'] = (exemployment['total_employed_moe2'])**(1/2)
exemployment['unemployed_moe'] = (exemployment['unemployed_moe2'])**(1/2)
exemployment['not_in_labor_moe'] = (exemployment['not_in_labor_moe2'])**(1/2)
exemployment['in_labor_moe'] = (exemployment['in_labor_moe2'])**(1/2)

#percentages
exemployment['pct_employed'] = 100*(exemployment['total_employed']/exemployment['pop'])
exemployment['pct_unemployed'] = 100*(exemployment['unemployed']/exemployment['pop'])
exemployment['pct_not_in_labor'] = 100*(exemployment['not_in_labor']/exemployment['pop'])
exemployment['pct_in_labor'] = 100*(exemployment['in_labor']/exemployment['pop'])

In [117]:
#generate proportion MOEs
exemployment['pct_employed_moe'] = 100*(prop_MOE(exemployment['total_employed'], exemployment['pop'], exemployment['total_employed_moe'], exemployment['pop_moe']))
exemployment['pct_unemployed_moe'] = 100*(prop_MOE(exemployment['unemployed'], exemployment['pop'], exemployment['unemployed_moe'], exemployment['pop_moe']))
exemployment['pct_not_in_labor_moe'] = 100*(prop_MOE(exemployment['not_in_labor'], exemployment['pop'], exemployment['not_in_labor_moe'], exemployment['pop_moe']))
exemployment['pct_in_labor_moe'] = 100*(prop_MOE(exemployment['in_labor'], exemployment['pop'], exemployment['in_labor_moe'], exemployment['pop_moe']))


In [118]:
#set variables to keep 
keep_vars = ['pop', 'total_employed', 'unemployed', 'not_in_labor', 
             'pop_moe', 'total_employed_moe', 'unemployed_moe', 'not_in_labor_moe', 
            'pct_employed', 'pct_unemployed', 'pct_not_in_labor', 
            'pct_employed_moe', 'pct_unemployed_moe', 'pct_not_in_labor_moe', 'in_labor',
            'in_labor_moe', 'pct_in_labor', 'pct_in_labor_moe']

exemployment = exemployment[keep_vars]
exemployment = exemployment.round(2)

In [119]:
#export data to csv
export_path = cwd+'/Output/python_output/employment.csv'
exemployment = exemployment.to_csv(export_path)

In [61]:
#####################################
##Section 3: Vehicle Availability
#####################################
vehicles['total'] = vehicles["Estimate; Total:"].astype(int)
vehicles['total_moe'] = vehicles["Margin of Error; Total:"].astype(int)
vehicles['none'] = vehicles["Estimate; Total: - No vehicle available"].astype(int)
vehicles['none_moe'] = vehicles["Margin of Error; Total: - No vehicle available"].astype(int)
vehicles['one'] = vehicles["Estimate; Total: - 1 vehicle available"].astype(int)
vehicles['one_moe'] = vehicles["Margin of Error; Total: - 1 vehicle available"].astype(int)
vehicles['two'] = vehicles["Estimate; Total: - 2 vehicles available"].astype(int)
vehicles['two_moe'] = vehicles["Margin of Error; Total: - 2 vehicles available"].astype(int)
vehicles['three'] = vehicles["Estimate; Total: - 3 vehicles available"].astype(int)
vehicles['three_moe'] = vehicles["Margin of Error; Total: - 3 vehicles available"].astype(int)
vehicles['four'] = vehicles["Estimate; Total: - 4 or more vehicles available"].astype(int)
vehicles['four_moe'] = vehicles["Margin of Error; Total: - 4 or more vehicles available"].astype(int)


In [62]:
#Create employment standard MOEs
vehicles['total_moe2'] = (vehicles['total_moe'])**2
vehicles['none_moe2'] = (vehicles['none_moe'])**2
vehicles['one_moe2'] = (vehicles['one_moe'])**2
vehicles['two_moe2'] = (vehicles['two_moe'])**2
vehicles['three_moe2'] = (vehicles['three_moe'])**2
vehicles['four_moe2'] = (vehicles['four_moe'])**2


In [63]:
#list data to include in sum
vehicles_list = ['total', 'none', 'one', 'two', 'three', 'four',
            'total_moe2', 'none_moe2', 'one_moe2', 'two_moe2', 'three_moe2', 'four_moe2']


In [64]:
#Sum data 

exvehicles = vehicles.groupby('Fruitvale')[vehicles_list].sum()

#new MOEs
exvehicles['total_moe'] = (exvehicles['total_moe2'])**(1/2)
exvehicles['none_moe'] = (exvehicles['none_moe2'])**(1/2)
exvehicles['one_moe'] = (exvehicles['one_moe2'])**(1/2)
exvehicles['two_moe'] = (exvehicles['two_moe2'])**(1/2)
exvehicles['three_moe'] = (exvehicles['three_moe2'])**(1/2)
exvehicles['four_moe'] = (exvehicles['four_moe2'])**(1/2)

#percentages
exvehicles['pct_none'] = 100*(exvehicles['none']/exvehicles['total'])
exvehicles['pct_one'] = 100*(exvehicles['one']/exvehicles['total'])
exvehicles['pct_two'] = 100*(exvehicles['two']/exvehicles['total'])
exvehicles['pct_three'] = 100*(exvehicles['three']/exvehicles['total'])
exvehicles['pct_four'] = 100*(exvehicles['four']/exvehicles['total'])



In [65]:
#generate proportion MOEs
exvehicles['pct_none_moe'] = 100*(prop_MOE(exvehicles['none'], exvehicles['total'], exvehicles['none_moe'], exvehicles['total_moe']))
exvehicles['pct_one_moe'] = 100*(prop_MOE(exvehicles['one'], exvehicles['total'], exvehicles['one_moe'], exvehicles['total_moe']))
exvehicles['pct_two_moe'] = 100*(prop_MOE(exvehicles['two'], exvehicles['total'], exvehicles['two_moe'], exvehicles['total_moe']))
exvehicles['pct_three_moe'] = 100*(prop_MOE(exvehicles['three'], exvehicles['total'], exvehicles['three_moe'], exvehicles['total_moe']))
exvehicles['pct_four_moe'] = 100*(prop_MOE(exvehicles['four'], exvehicles['total'], exvehicles['four_moe'], exvehicles['total_moe']))



In [66]:
#set variables to keep 
keep_vars = ['pct_none', 'pct_one', 'pct_two', 'pct_three', 'pct_four', 
             'pct_none_moe', 'pct_one_moe', 'pct_two_moe', 'pct_three_moe', 'pct_four_moe']

exvehicles = exvehicles[keep_vars]

#change everything to rounded percents
exvehicles = exvehicles.round(2)

exvehicles = exvehicles.transpose()


In [67]:
#export data to csv
export_path = cwd+'/Output/python_output/vehicles.csv'
exvehicles = exvehicles.to_csv(export_path)

In [68]:
#############################
##Section 4: Time to Work
#############################

#Convert all employment variables to integers
time['pop'] = time["Estimate; Total:"].astype(int)
time['pop_moe'] = time["Margin of Error; Total:"].astype(int)
time['ten'] = time["Estimate; Total: - Less than 10 minutes"].astype(int)
time['ten_moe'] = time["Margin of Error; Total: - Less than 10 minutes"].astype(int)
time['fourteen'] = time["Estimate; Total: - 10 to 14 minutes"].astype(int)
time['fourteen_moe'] = time["Margin of Error; Total: - 10 to 14 minutes"].astype(int)
time['nineteen'] = time["Estimate; Total: - 15 to 19 minutes"].astype(int)
time['nineteen_moe'] = time["Margin of Error; Total: - 15 to 19 minutes"].astype(int)
time['twenty_four'] = time["Estimate; Total: - 20 to 24 minutes"].astype(int)
time['twenty_four_moe'] = time["Margin of Error; Total: - 20 to 24 minutes"].astype(int)
time['twenty_nine'] = time["Estimate; Total: - 25 to 29 minutes"].astype(int)
time['twenty_nine_moe'] = time["Margin of Error; Total: - 25 to 29 minutes"].astype(int)
time['thirty_four'] = time["Estimate; Total: - 30 to 34 minutes"].astype(int)
time['thirty_four_moe'] = time["Margin of Error; Total: - 30 to 34 minutes"].astype(int)
time['forty_four'] = time["Estimate; Total: - 35 to 44 minutes"].astype(int)
time['forty_four_moe'] = time["Margin of Error; Total: - 35 to 44 minutes"].astype(int)
time['fifty_nine'] = time["Estimate; Total: - 45 to 59 minutes"].astype(int)
time['fifty_nine_moe'] = time["Margin of Error; Total: - 45 to 59 minutes"].astype(int)
time['sixty'] = time["Estimate; Total: - 60 or more minutes"].astype(int)
time['sixty_moe'] = time["Margin of Error; Total: - 60 or more minutes"].astype(int)





In [69]:
#Create employment standard MOEs
time['pop_moe2'] = (time['pop_moe'])**2
time['ten_moe2'] = (time['ten_moe'])**2
time['fourteen_moe2'] = (time['fourteen_moe'])**2
time['nineteen_moe2'] = (time['nineteen_moe'])**2
time['twenty_four_moe2'] = (time['twenty_four_moe'])**2
time['twenty_nine_moe2'] = (time['twenty_nine_moe'])**2
time['thirty_four_moe2'] = (time['thirty_four_moe'])**2
time['forty_four_moe2'] = (time['forty_four_moe'])**2
time['fifty_nine_moe2'] = (time['fifty_nine_moe'])**2
time['sixty_moe2'] = (time['sixty_moe'])**2


#create broader categories, their moes, and their moes squared
time['fifteen'] = time['ten'] + time['fourteen']
time['fifteen_moe'] = (time['ten_moe2'] + time['fourteen_moe2'])**(1/2) 
time['fifteen_moe2'] = (time['fifteen_moe'])**2

time['thirty'] = time['nineteen'] + time['twenty_four'] + time['twenty_nine']
time['thirty_moe'] = (time['nineteen_moe2'] + time['twenty_four_moe2'] + time['twenty_nine_moe2'])**(1/2) 
time['thirty_moe2'] = (time['thirty_moe'])**2

time['forty_five'] = time['thirty_four'] + time['forty_four']
time['forty_five_moe'] = (time['thirty_four_moe2'] + time['forty_four_moe2'])**(1/2) 
time['forty_five_moe2'] = (time['forty_five_moe'])**2



In [70]:
#list data to include in sum
times = ['pop', 'fifteen', 'thirty', 'forty_five', 'fifty_nine', 'sixty',
                 'pop_moe2', 'fifteen_moe2', 'thirty_moe2','forty_five_moe2', 'fifty_nine_moe2', 
                 'sixty_moe2']


In [71]:
#Sum data 

extime = time.groupby('Fruitvale')[times].sum()


In [72]:
#new MOEs
extime['pop_moe'] = (extime['pop_moe2'])**(1/2)
extime['fifteen_moe'] = (extime['fifteen_moe2'])**(1/2)
extime['thirty_moe'] = (extime['thirty_moe2'])**(1/2)
extime['forty_five_moe'] = (extime['forty_five_moe2'])**(1/2)
extime['fifty_nine_moe'] = (extime['fifty_nine_moe2'])**(1/2)
extime['sixty_moe'] = (extime['sixty_moe2'])**(1/2)

In [73]:
#percentages
extime['pct_fifteen'] = 100*(extime['fifteen']/extime['pop'])
extime['pct_thirty'] = 100*(extime['thirty']/extime['pop'])
extime['pct_forty_five'] = 100*(extime['forty_five']/extime['pop'])
extime['pct_fifty_nine'] = 100*(extime['fifty_nine']/extime['pop'])
extime['pct_sixty'] = 100*(extime['sixty']/extime['pop'])

In [74]:
#generate proportion MOEs
extime['pct_fifteen_moe'] = 100*(prop_MOE(extime['fifteen'], extime['pop'], extime['fifteen_moe'], extime['pop_moe']))
extime['pct_thirty_moe'] = 100*(prop_MOE(extime['thirty'], extime['pop'], extime['thirty_moe'], extime['pop_moe']))
extime['pct_forty_five_moe'] = 100*(prop_MOE(extime['forty_five'], extime['pop'], extime['forty_five_moe'], extime['pop_moe']))
extime['pct_fifty_nine_moe'] = 100*(prop_MOE(extime['fifty_nine'], extime['pop'], extime['fifty_nine_moe'], extime['pop_moe']))
extime['pct_sixty_moe'] = 100*(prop_MOE(extime['sixty'], extime['pop'], extime['sixty_moe'], extime['pop_moe']))

extime

,pop,fifteen,thirty,forty_five,fifty_nine,sixty,pop_moe2,fifteen_moe2,thirty_moe2,forty_five_moe2,...,pct_fifteen,pct_thirty,pct_forty_five,pct_fifty_nine,pct_sixty,pct_fifteen_moe,pct_thirty_moe,pct_forty_five_moe,pct_fifty_nine_moe,pct_sixty_moe
Fruitvale,,,,,,,,,,,,,,,,,,,,,
False,662527,131123,218154,161850,73069,78331,23759708,6760498.0,11426567.0,7854463.0,...,19.791344,32.927564,24.429193,11.028834,11.823065,0.364439,0.449035,0.382932,0.266233,0.284465
True,21409,3802,8318,5122,1897,2270,1341146,299779.0,500883.0,291807.0,...,17.758886,38.852819,23.924518,8.860759,10.603017,2.370158,2.551678,2.166035,1.297084,1.610539


In [75]:
#list data to include in export
time_vars = ['pop', 'pct_fifteen', 'pct_thirty', 'pct_forty_five', 'pct_fifty_nine', 'pct_sixty',
            'pct_fifteen_moe', 'pct_thirty_moe', 'pct_forty_five_moe', 'pct_fifty_nine_moe', 'pct_sixty_moe']

In [76]:
extime = extime[time_vars]

#change everything to rounded percents
extime = extime.round(2)

In [77]:
#export data to csv
export_path = cwd+'/Output/python_output/time.csv'
extime = extime.to_csv(export_path)

In [78]:
#######################################
##Section 5 choice C: Household Income
#######################################

#Convert all employment variables to integers
income['total'] = income["Estimate; Total:"].astype(int)
income['total_moe'] = income["Margin of Error; Total:"].astype(int)
income['ten'] = income["Estimate; Total: - Less than $10,000"].astype(int)
income['ten_moe'] = income["Margin of Error; Total: - Less than $10,000"].astype(int)
income['fifteen'] = income["Estimate; Total: - $10,000 to $14,999"].astype(int)
income['fifteen_moe'] = income["Margin of Error; Total: - $10,000 to $14,999"].astype(int)
income['twenty'] = income["Estimate; Total: - $15,000 to $19,999"].astype(int)
income['twenty_moe'] = income["Margin of Error; Total: - $15,000 to $19,999"].astype(int)
income['twenty_five'] = income["Estimate; Total: - $20,000 to $24,999"].astype(int)
income['twenty_five_moe'] = income["Margin of Error; Total: - $20,000 to $24,999"].astype(int)
income['thirty'] = income["Estimate; Total: - $25,000 to $29,999"].astype(int)
income['thirty_moe'] = income["Margin of Error; Total: - $25,000 to $29,999"].astype(int)
income['thirty_five'] = income["Estimate; Total: - $30,000 to $34,999"].astype(int)
income['thirty_five_moe'] = income["Margin of Error; Total: - $30,000 to $34,999"].astype(int)
income['forty'] = income["Estimate; Total: - $35,000 to $39,999"].astype(int)
income['forty_moe'] = income["Margin of Error; Total: - $35,000 to $39,999"].astype(int)
income['forty_five'] = income["Estimate; Total: - $40,000 to $44,999"].astype(int)
income['forty_five_moe'] = income["Margin of Error; Total: - $40,000 to $44,999"].astype(int)
income['fifty'] = income["Estimate; Total: - $45,000 to $49,999"].astype(int)
income['fifty_moe'] = income["Margin of Error; Total: - $45,000 to $49,999"].astype(int)
income['sixty'] = income["Estimate; Total: - $50,000 to $59,999"].astype(int)
income['sixty_moe'] = income["Margin of Error; Total: - $50,000 to $59,999"].astype(int)
income['seventy_five'] = income["Estimate; Total: - $60,000 to $74,999"].astype(int)
income['seventy_five_moe'] = income["Margin of Error; Total: - $60,000 to $74,999"].astype(int)
income['hundred'] = income["Estimate; Total: - $75,000 to $99,999"].astype(int)
income['hundred_moe'] = income["Margin of Error; Total: - $75,000 to $99,999"].astype(int)
income['hundred25'] = income["Estimate; Total: - $100,000 to $124,999"].astype(int)
income['hundred25_moe'] = income["Margin of Error; Total: - $100,000 to $124,999"].astype(int)
income['hundred50'] = income["Estimate; Total: - $125,000 to $149,999"].astype(int)
income['hundred50_moe'] = income["Margin of Error; Total: - $125,000 to $149,999"].astype(int)
income['twohundred'] = income["Estimate; Total: - $150,000 to $199,999"].astype(int)
income['twohundred_moe'] = income["Margin of Error; Total: - $150,000 to $199,999"].astype(int)
income['more_twohundred'] = income["Estimate; Total: - $200,000 or more"].astype(int)
income['more_twohundred_moe'] = income["Margin of Error; Total: - $200,000 or more"].astype(int)



In [79]:
#keep certain variables
income_list = ['total_moe', 'ten_moe', 'fifteen_moe', 
               'twenty_moe', 'twenty_five_moe', 'thirty_moe', 
              'thirty_five_moe', 'forty_moe', 'forty_five_moe', 
              'fifty_moe', 'sixty_moe', 'seventy_five_moe', 
              'hundred_moe', 'hundred25_moe', 'hundred50_moe', 
              'twohundred_moe', 'more_twohundred_moe']

#convert MOEs to 95% confidence level
def convert(column):
    return column*(1.96/1.645)

#square columns
def square(column):
    return column**2

for item in income_list:
    income[item] = convert(income[item])
    income[item+'2']= square(income[item])



In [80]:
#Create aggregate columns and MOEs

income['less_25'] = income['ten'] + income['fifteen'] + income['twenty']+ income['twenty_five']
income['less_25_moe'] = (income['ten_moe2'] + income['fifteen_moe2'] + income['twenty_moe2']+ income['twenty_five_moe2'])**(1/2) 
income['less_25_moe2'] = (income['less_25_moe'])**2

income['25_50'] = income['thirty'] + income['thirty_five'] + income['forty']+ income['forty_five'] + income['fifty']
income['25_50_moe'] = (income['thirty_moe2'] + income['thirty_five_moe2'] + income['forty_moe2']+ income['forty_five_moe2'] + income['fifty_moe2'])**(1/2) 
income['25_50_moe2'] = (income['25_50_moe'])**2

income['50_75'] = income['sixty'] + income['seventy_five'] 
income['50_75_moe'] = (income['sixty_moe2'] + income['seventy_five_moe2'] )**(1/2) 
income['50_75_moe2'] = (income['25_50_moe'])**2

income['100_plus'] = income['hundred25'] + income['hundred50'] + income['twohundred'] + income['more_twohundred']
income['100_plus_moe'] = (income['hundred25_moe2'] + income['hundred50_moe2'] + income['twohundred_moe2'] + income['more_twohundred_moe2'])**(1/2) 
income['100_plus_moe2'] = (income['100_plus_moe'])**2



In [81]:
#Sum data 

#list variables to include in sum
incomes = ['total', 'less_25', '25_50', '50_75', 'hundred', '100_plus', 
          'total_moe2', 'less_25_moe2', '25_50_moe2', '50_75_moe2', 
          'hundred_moe2', '100_plus_moe2']

exincome = income.groupby('Fruitvale')[incomes].sum()


In [82]:
#Create new MOEs after sum
moe_list = ['total_moe2', 'less_25_moe2', '25_50_moe2', '50_75_moe2', 
          'hundred_moe2', '100_plus_moe2']

def sq_root(column):
    return column**(1/2)

for item in moe_list:
    exincome[item.strip('2')] = sq_root(exincome[item])
    
#rename one item from previous list
exincome.rename(columns={'5_50_moe': '25_50_moe'}, inplace = True)

In [83]:
#Create Percentages
pct_list = ['total', 'less_25', '25_50', '50_75', 'hundred', '100_plus']

def pct(column):
    return 100*(exincome[column]/exincome['total'])

for item in pct_list:
    exincome['pct_'+item] = pct(item)

In [84]:
#Create New Proportions MOEs
exincome['pct_less_25_moe'] = 100*(prop_MOE(exincome['less_25'], exincome['total'], exincome['less_25_moe'], exincome['total_moe']))
exincome['pct_25_50_moe'] = 100*(prop_MOE(exincome['25_50'], exincome['total'], exincome['25_50_moe'], exincome['total_moe']))
exincome['pct_50_75_moe'] = 100*(prop_MOE(exincome['50_75'], exincome['total'], exincome['50_75_moe'], exincome['total_moe']))
exincome['pct_hundred_moe'] = 100*(prop_MOE(exincome['hundred'], exincome['total'], exincome['hundred_moe'], exincome['total_moe']))
exincome['pct_100_plus_moe'] = 100*(prop_MOE(exincome['100_plus'], exincome['total'], exincome['100_plus_moe'], exincome['total_moe']))


In [85]:
#list data to include in export
export = ['total', 'pct_less_25', 'pct_25_50', 'pct_50_75', 'pct_hundred', 'pct_100_plus',
         'pct_less_25_moe', 'pct_25_50_moe', 'pct_50_75_moe', 'pct_hundred_moe', 
         'pct_100_plus_moe']

exincome = exincome[export]

#change everything to rounded percents
exincome = exincome.round(2)

In [86]:
#export data to csv
export_path = cwd+'/Output/python_output/income.csv'
exincome = exincome.to_csv(export_path)